In [1]:
'''
a.Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.
b.Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.
c.Tune the hyperparameters using cross-validation and see what precision you can achieve.
d.Now try adding Batch Normalization and compare the learning curves: is it converging faster than before? Does it produce a better model?
e.Is the model overfitting the training set? Try adding dropout to every layer and try again. Does it help?'''



import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Load MNIST dataset
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Split the data for digits 0 to 4
X_train_04 = X_train_full[y_train_full < 5]
y_train_04 = y_train_full[y_train_full < 5]

# Preprocess the data
X_train_04 = X_train_04 / 255.0
X_test = X_test / 255.0

# Build a DNN with five hidden layers, He initialization, and ELU activation
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
for _ in range(5):
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal', activation='elu'))
model.add(keras.layers.Dense(5, activation='softmax'))

# Using Adam optimization and early stopping
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('my_model.h5', save_best_only=True)

# Train the model
model.fit(X_train_04, y_train_04, epochs=100, validation_split=0.2,
          callbacks=[early_stopping_cb, model_checkpoint_cb])

# Load the best model
model = keras.models.load_model('my_model.h5')

# Hyperparameter tuning using cross-validation
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    for _ in range(5):
        model.add(keras.layers.Dense(100, kernel_initializer='he_normal', activation='elu'))
    model.add(keras.layers.Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

kfold = KFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(build_model(), X_train_04, y_train_04, cv=kfold)
print("Cross-Validation Mean Precision:", scores.mean())

# Add Batch Normalization
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
for _ in range(5):
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(5, activation='softmax'))

# Train the model with Batch Normalization
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_04, y_train_04, epochs=100, validation_split=0.2,
          callbacks=[early_stopping_cb, model_checkpoint_cb])

# Add Dropout to every layer
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
for _ in range(5):
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
    model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(5, activation='softmax'))

# Train the model with Dropout
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_04, y_train_04, epochs=100, validation_split=0.2,
          callbacks=[early_stopping_cb, model_checkpoint_cb])


ImportError: initialization failed

In [ ]:
'''
2.Transfer learning.
a.Create a new DNN that reuses all the pretrained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a new one.
b.Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision?
c.Try caching the frozen layers, and train the model again: how much faster is it now?
d.Try again reusing just four hidden layers instead of five. Can you achieve a higher precision?
e.Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

'''

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Filter out digits 5 to 9
train_mask = (train_labels >= 5)
test_mask = (test_labels >= 5)
train_images = train_images[train_mask]
train_labels = train_labels[train_mask] - 5  # Map labels to 0-4
test_images = test_images[test_mask]
test_labels = test_labels[test_mask] - 5

# Prepare the data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Load a pre-trained model and freeze its layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in base_model.layers:
    layer.trainable = False

# Create a new DNN on top of the pre-trained model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the new DNN on limited data
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=10, batch_size=32)

# Measure the time it takes for training
import time
start_time = time.time()
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=10, batch_size=32)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_accuracy}")

# Try caching the frozen layers and train again
model.trainable = True
for layer in base_model.layers[:100]:  # Cache the first 100 layers
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=10, batch_size=32)
end_time = time.time()
training_time_cached = end_time - start_time
print(f"Training time with cached layers: {training_time_cached} seconds")

# Try reusing just four hidden layers
model.layers[100].trainable = True  # Unfreeze the fourth layer

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=10, batch_size=32)
end_time = time.time()
training_time_4_layers = end_time - start_time
print(f"Training time with 4 layers: {training_time_4_layers} seconds")

# Unfreeze the top two hidden layers and continue training
for layer in model.layers[:102]:  # Unfreeze top two hidden layers
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=10, batch_size=32)
end_time = time.time()
training_time_unfreeze = end_time - start_time
print(f"Training time with unfrozen top 2 layers: {training_time_unfreeze} seconds")


In [ ]:
'''
3.Pretraining on an auxiliary task.
a.In this exercise you will build a DNN that compares two MNIST digit images and predicts whether they represent the same digit or not. Then you will reuse the lower layers of this network to train an MNIST classifier using very little training data. Start by building two DNNs (let’s call them DNN A and B), both similar to the one you built earlier but without the output layer: each DNN should have five hidden layers of 100 neurons each, He initialization, and ELU activation. Next, add one more hidden layer with 10 units on top of both DNNs. To do this, you should use TensorFlow’s concat() function with axis=1 to concatenate the outputs of both DNNs for each instance, then feed the result to the hidden layer. Finally, add an output layer with a single neuron using the logistic activation function.
b.Split the MNIST training set in two sets: split #1 should containing 55,000 images, and split #2 should contain contain 5,000 images. Create a function that generates a training batch where each instance is a pair of MNIST images picked from split #1. Half of the training instances should be pairs of images that belong to the same class, while the other half should be images from different classes. For each pair, the training label should be 0 if the images are from the same class, or 1 if they are from different classes.
c.Train the DNN on this training set. For each image pair, you can simultaneously feed the first image to DNN A and the second image to DNN B. The whole network will gradually learn to tell whether two images belong to the same class or not.
d.Now create a new DNN by reusing and freezing the hidden layers of DNN A and adding a softmax output layer on top with 10 neurons. Train this network on split #2 and see if you can achieve high performance despite having only 500 images per class.
'''

#a. Building two DNNs (DNN A and B) with shared lower layers:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model

# Define a function to create a DNN with 5 hidden layers, He initialization, and ELU activation
def create_dnn():
    model_input = Input(shape=(28*28,))
    hidden = model_input
    for _ in range(5):
        hidden = Dense(100, activation='elu', kernel_initializer='he_normal')(hidden)
    return Model(inputs=model_input, outputs=hidden)

# Create DNN A and B
dnn_a = create_dnn()
dnn_b = create_dnn()

# Create a layer to concatenate the outputs of DNN A and B
concatenated = Concatenate(axis=1)([dnn_a.output, dnn_b.output])

# Add a hidden layer with 10 units
hidden_layer = Dense(10, activation='elu', kernel_initializer='he_normal')(concatenated)

# Add an output layer with a single neuron using logistic activation
output_layer = Dense(1, activation='sigmoid')(hidden_layer)

# Create the final model
model = Model(inputs=[dnn_a.input, dnn_b.input], outputs=output_layer)

# Compile the model with an appropriate optimizer and loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
#b. Creating a function to generate training batches:
import numpy as np

# Function to generate training batches
def generate_training_batch(split1, batch_size=32):
    batch_images = []
    batch_labels = []
    
    for _ in range(batch_size // 2):
        # Select a pair of images from split1
        pair_indices = np.random.choice(len(split1), 2, replace=False)
        image1, image2 = split1[pair_indices[0]], split1[pair_indices[1]]
        
        # Label 0 if the images are from the same class, 1 if they are from different classes
        if pair_indices[0] // 5500 == pair_indices[1] // 5500:
            label = 0
        else:
            label = 1
        
        batch_images.append([image1, image2])
        batch_labels.append(label)
    
    return np.array(batch_images), np.array(batch_labels)


In [ ]:
# c. Training the DNN:
# Assuming you have the MNIST dataset loaded as mnist_data
from sklearn.model_selection import train_test_split

# Split the MNIST training set into two sets
split1, split2 = train_test_split(mnist_data, test_size=5000, random_state=42)

# Train the DNN on the training set
batch_size = 32
epochs = 10

for epoch in range(epochs):
    for iteration in range(len(split1) // batch_size):
        batch_images, batch_labels = generate_training_batch(split1, batch_size)
        model.train_on_batch([batch_images[:,0], batch_images[:,1]], batch_labels)


In [ ]:
# d. Creating a new DNN by reusing and freezing hidden layers of DNN A and training it on split 
# Remove the last output layer from DNN A
dnn_a.pop()

# Freeze the layers of DNN A
for layer in dnn_a.layers:
    layer.trainable = False

# Add a new output layer with 10 neurons (softmax for classification)
output_layer_split2 = Dense(10, activation='softmax')(dnn_a.layers[-1].output)

# Create the new model
model_split2 = Model(inputs=dnn_a.input, outputs=output_layer_split2)

# Compile the new model for classification
model_split2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have split #2 as X_split2 and y_split2
model_split2.fit(X_split2, y_split2, epochs=10)
